# Machine Learning 3253 - Project Assignment - "Going to the Movies"

## Team Members
    
* Craig Barbisan
* Nisha Choondassery
* Mark Hubbard
* Itay Segal

## Project Overview

Using the "Movies Dataset"  from Kaggle, this project will create an optimal model for predicting a movie's rating (*__TO_REVIEW__*)

### The movies dataset
The dataset for this project is originally sourced from https://www.kaggle.com/rounakbanik/the-movies-dataset.

Credit: Rounak Banik

This dataset is an ensemble of data collected from TMDB and GroupLens.
* The Movie Details (i.e. Metadata), Credits and Keywords have been collected from the TMDB Open API.
* The Movie Links and Ratings have been obtained from the Official GroupLens website.

The following spreadsheets are used by this project:

* __movies_metadata.csv__: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

* __credits.csv__: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

* __keywords.csv__:  Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

The Full MovieLens dataset consisting of 26 million ratings and 750,000 tag applications from 270,000 users on all the 45,000 movies in this dataset can be accessed at https://grouplens.org/datasets/movielens/latest/


#### Frame the Problem and Look at the Big Picture (*TO_REVIEW*)
Define the objective in business terms.

How will your solution be used?

What are the current solutions/workarounds (if any)?

How should you frame this problem (supervised/unsupervised, online/offline, etc.)?

How should performance be measured?

Is the performance measure aligned with the business objective?

What would be the minimum performance needed to reach the business objective?

What are comparable problems? Can you reuse experience or tools?

Is human expertise available?

How would you solve the problem manually?

List the assumptions you (or others) have made so far.

Verify assumptions if possible.

## Notebook Contents

This notebook will explore the data, evaluate some models and draw conclusions. It is divided into the following main sections:

0. Setup the environment - seting up the notebook environment.
1. Get the data - loading the data set into the notebook.
2. Explore the data - exploring the raw movies data.
3. Prepare the data - data cleansing, feature selection\reduction\engineering and standardization

2. Model - training and testing various models.
3. Analysis - analyzing the model results.
4. Summary - summarizing the observations and conclusions.

## 0. Setup the environment

### Initialize libraries

In [ ]:
# import the basic libraries
import os
import numpy as np
import pandas as pd
import json
import ast
import tarfile

In [ ]:
# scatter matrix plotting
from pandas.plotting import scatter_matrix

# enable advanced plots
import seaborn as sns

In [ ]:
# for reading and writing ZIP files
from zipfile import ZipFile
# to retrieve from a url
from urllib.request import urlretrieve
# to parse URLs into components
from urllib.parse import urlparse

In [ ]:
# import sklearn libraries

# feature extraction and decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF

# feature selection and reduction
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
# feature clustering
from sklearn.cluster import KMeans

# pipeline processing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

# data splitting
from sklearn.model_selection import train_test_split

# classifier models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# model evaluation
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

### Basic settings

In [ ]:
# make this notebook's output stable across runs
SEED = 42
np.random.seed(SEED)

In [ ]:
# ensure full display for dataframe content
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('precision', 5)
pd.set_option('large_repr', 'truncate')
pd.set_option('display.max_colwidth', -1)
pd.set_option('colheader_justify', 'left')

In [ ]:
# enable basic plots with pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
# advanced plots settings
sns.set(style='darkgrid')
sns.set(font_scale=1.2)

In [ ]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

### Constants

In [ ]:
# specify file names and locations
URL_DROP_DOMAIN    = 'https://www.dropbox.com'  # data is stored on dropbox due to size limitations of GitHub
URL_DROP_PATH      = '/s/89uv5kntgiolkno/the-movies-dataset.zip?dl=1'

URL_GIT_DOMAIN    = 'https://raw.githubusercontent.com'  # GitHub domain
URL_GIT_PROJECT_PATH      = '/itayse10/GoingToMovies/master'  # GitHub project path

PROJECT_LOCAL_ROOT_DIR = 'movies'
PROJECT_LOCAL_DATA_DIR  = os.path.join(PROJECT_LOCAL_ROOT_DIR,'datasets')
PROJECT_LOCAL_RESOURCE_DIR  = os.path.join(PROJECT_LOCAL_ROOT_DIR,'resources')
PROJECT_LOCAL_CHECKPOINT_DIR  = os.path.join(PROJECT_LOCAL_ROOT_DIR,'checkpoints')

PROJECT_OUTPUT_DIR = '/content/movies/output'
PROJECT_FILE       = 'the-movies-dataset.zip'

## 1. Get the data

### Data retrieval and storage functions

In [ ]:
# function: download files from the internet and optionally unzip them
def fetch_data(url, file, local_path, zip=False):
    if not os.path.isdir(local_path):
        os.makedirs(local_path)
    
    remote_file = url
    local_file  = os.path.join(local_path, file)
   
    print(remote_file)
    
    if not os.path.isfile(local_file):
        print('Downloading ' + file + '...')
        
        download_from_Web(remote_file, local_file)

        print('Download complete.')
        
    else:
        print('Already downloaded.')

    if zip:
       unzip_file(local_file)

In [ ]:
# unzip a file
def unzip_file(file):
 
    zip_path = file[:-4]
  
    if (os.path.isdir(zip_path)):
        print('Already extracted.')
    else:
        print('Extracting...')
        zfile = ZipFile(file, 'r')
        print(zfile.infolist())
        zfile.extractall(zip_path)
        zfile.close()
        print('Extraction complete.')

In [ ]:
# download an entire file from a public Web site
def download_from_Web(remote_file, local_file):
    urlretrieve(remote_file, local_file)

In [ ]:
# save a checkpoint for a dataset locally
def save_checkpoint(df, df_name, checkpoint_identifier):
    current_checkpoint = 'checkpoint_' + checkpoint_identifier
    directory_path = os.path.join(PROJECT_LOCAL_CHECKPOINT_DIR,current_checkpoint)
    if not os.path.isdir(directory_path):
        os.makedirs(directory_path)
        print('Creating new directory: {}'.format(directory_path))
    target_file = df_name + '.csv'
    file_path = os.path.join(directory_path, target_file)
    df.to_csv(file_path, encoding='utf-8', index=False )
    print('File {} saved in {}'.format(target_file, directory_path))

In [ ]:
# restore a checkpoint into a dataframe object
def restore_checkpoint(df_name,checkpoint_identifier):
    current_checkpoint = 'checkpoint_' + checkpoint_identifier
    directory_path = os.path.join(PROJECT_LOCAL_CHECKPOINT_DIR,current_checkpoint)
    if not os.path.isdir(directory_path):
        print('Directory for checkpoint does not exist: {}'.format(directory_path))
        exit()
    target_file = df_name + '.csv'
    file_path = os.path.join(directory_path, target_file)
    if not os.path.isfile(file_path):
        print('File for checkpoint does not exist: {}'.format(file_path))
        exit()
    df = pd.read_csv(file_path,   encoding='utf-8'   )
    return df

### Data retrieval - load the data sets

In [ ]:
# download the dataset file
url = URL_DROP_DOMAIN + URL_DROP_PATH
fetch_data(url, PROJECT_FILE, PROJECT_LOCAL_DATA_DIR, zip=True)

### Create initial data frames
Create dataframes for each of the spreadsheets.

As part of loading the dataframes, convert null values to NaN on the fly (via pd.read_csv).

By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’.

In [ ]:
# initialize the dataframes (and convert null fields on the fly)
zip_path = os.path.join(PROJECT_LOCAL_DATA_DIR, PROJECT_FILE[:-4])

metadata_file = os.path.join(zip_path, 'movies_metadata.csv')
credits_file  = os.path.join(zip_path, 'credits.csv')
plot_file     = os.path.join(zip_path, 'keywords.csv')

# load the metadata dataframe
metadata = pd.read_csv(metadata_file,
                     dtype = 'unicode',
                     na_values = ['no info', '.']
                    )

# load the credits dataframe
credits = pd.read_csv(credits_file,
                      dtype = 'unicode',
                      na_values = ['no info', '.']
                     )

# load the plot dataframe
plot =  pd.read_csv(plot_file,
                    dtype = 'unicode',
                    na_values = ['no info', '.']
                   )

## 2. Explore the data

In [ ]:
# make copies of data for EDA
metadata_copy = metadata.copy()
metadata_copy.name = 'metadata'

credits_copy = credits.copy()
credits_copy.name = 'credit'

plot_copy = plot.copy()
plot_copy.name = 'plot'

### Data exploration functions

In [ ]:
# basic schema analysis
def quick_schema_analysis(df):
    if not hasattr(df, 'name'):
        df.name = '' # set an empty name
    print("Basic Schema Analysis for dataframe=" + df.name)
    print("************************************************")
    
    print("Rows and Columns:")
    print(df.shape)
    print(df.info())
    print("\n")
    
    print("Null Values - percentage:")
    print((1 - df.count()/len(df.index)) * 100)
    print("\n")
    
    print("Null Values - count:")
    print(df.isnull().sum())
    print("\n")

In [ ]:
# basic data analysis
def quick_data_analysis(df):
    if not hasattr(df, 'name'):
        df.name = '' # set an empty name
    print("Basic Data Analysis for dataframe=" + df.name)
    print(df.shape)

In [ ]:
# review column results
def assess_columns(df, columns, categories=False):
    for column in columns:
        print('Info for column: {}'.format(column))
        num_empty = df[column].isnull().sum()
        print('Number of null entries: {}'.format(num_empty))
        if categories:
            num_unique = df[column].unique()
            print('Unique values: {}'.format(num_unique))
    print(df[columns].head(10))

### Explore the "metadata" data

#### Potential features

* __adult__: Indicates if the movie is X-Rated or Adult.
* __belongs_to_collection__: A stringified dictionary that gives information on the movie series the particular film belongs to.
* __budget__: The budget of the movie in dollars.
* __genres__: A stringified list of dictionaries that list out all the genres associated with the movie.
* __homepage__: The Official Homepage of the move.
* __id__: The ID of the movie.
* __imdb_id__: The IMDB ID of the movie.
* __original_language__: The language in which the movie was originally shot in.
* __original_title__: The original title of the movie.
* __overview__: A brief blurb of the movie.
* __popularity__: The Popularity Score assigned by TMDB.
* __poster_path__: The URL of the poster image.
* __production_companies__: A stringified list of production companies involved with the making of the movie.
* __production_countries__: A stringified list of countries where the movie was shot/produced in.
* __release_date__: Theatrical Release Date of the movie.
* __revenue__: The total revenue of the movie in dollars.
* __runtime__: The runtime of the movie in minutes.
* __spoken_languages__: A stringified list of spoken languages in the film.
* __status__: The status of the movie (Released, To Be Released, Announced, etc.)
* __tagline__: The tagline of the movie.
* __title__: The Official Title of the movie.
* __video__: Indicates if there is a video present of the movie with TMDB.
* __vote_average__: The average rating of the movie.
* __vote_count__: The number of votes by users, as counted by TMDB.

In [ ]:
# quick review of the metadata data
quick_schema_analysis(metadata_copy)
quick_data_analysis(metadata_copy)

In [ ]:
# examine the first 10 rows
metadata_copy.head(10)

In [ ]:
# additional column info - counts, unique values, etc
metadata_copy.describe()

#### Assess columns

In [ ]:
assess_columns(metadata_copy, ['release_date'])

In [ ]:
assess_columns(metadata_copy, ['budget','revenue'])

In [ ]:
# asses json columns
assess_columns(metadata_copy, ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages'])

In [ ]:
# assess numerical columns
assess_columns(metadata_copy, ['runtime','vote_average','vote_count'])

### Explore the "credits" data

In [ ]:
# quick review of the credits data
quick_schema_analysis(credits_copy)
quick_data_analysis(credits_copy)

In [ ]:
# examine the first 10 rows
credits_copy.head(10)

#### Assess columns

In [ ]:
assess_columns(credits_copy, ['cast', 'crew'])

### Explore the "plot" data

In [ ]:
# quick review of the plot data
quick_schema_analysis(plot_copy)
quick_data_analysis(plot_copy)

In [ ]:
# examine the first 10 rows
plot_copy.head(10)

#### Assess columns

In [ ]:
assess_columns(plot_copy,['keywords'])

## 3. Prepare the data

### Merge the 3 raw data sets

In [ ]:
# compare the shapes of all the dataframes
print(metadata.shape)
print(credits.shape)
print(plot.shape)

In [ ]:
# initialize the merged dataframe "movies"
movies = metadata.copy()

In [ ]:
# merge the other data frames
movies = movies.merge(credits, on=["id"])
movies = movies.merge(plot, on=['id'])

In [ ]:
# check the new data shape
print(movies.shape)

In [ ]:
# explore the movies data
movies.head(5)

### Create data checkpoint - checkpoint_1
Seperate data sets at their raw state

In [ ]:
# save checkpoint of the dataframes
save_checkpoint(metadata, 'metadata', '1')
save_checkpoint(credits, 'credits', '1')
save_checkpoint(plot, 'plot', '1')
save_checkpoint(movies, 'movies', '1')

In [ ]:
# restore data into datasets - in case you need to cotinue the code from this point (disable previous cell)
# metadata = restore_checkpoint('metadata','1')
# credits = restore_checkpoint('credits','1')
# plot = restore_checkpoint('plot','1')
# movies = restore_checkpoint('movies','1')

### Data conversion functions

In [ ]:
# convert json to abstract syntax trees (ast)
# use ast because json data has single quotes in the csvs
# which is invalid for a json object (should be double quotes)
def convert_json_to_ast(df, json_columns):
    for column in json_columns:
        df[column] = df[column].apply(lambda x: np.nan if pd.isnull(x)
                                                else ast.literal_eval(str(x)))

In [ ]:
# convert the columns that contain the dictionary field with the name value in its ast 
def get_dict_val_from_ast(df,columns,field_name, fillna_str):
    for column in columns:
        df[column] = df[column].fillna(fillna_str) # first replace NaN with fillna_str
        df[column] = df[column].apply(lambda x: x[field_name] if isinstance(x, dict) else [])

In [ ]:
# convert the columns that contain the list field with the name value in its ast
def get_list_val_from_ast(df,columns,field_name, fillna_str=None, new_col_dict=None):
    for column in columns:
        if(fillna_str):
            df[column] = df[column].fillna(fillna_str) # first replace NaN with fillna_str
        new_column = column
        if(new_col_dict and column in new_col_dict):
            new_column = new_col_dict[column]
        df[new_column] = df[column].apply(lambda x: [i[field_name] for i in x] if isinstance(x, list) else []) 

In [ ]:
# convert to float
def cast_to_float(df, columns,d_cast=None):
    for column in columns:
        if(d_cast):
            df[column] = pd.to_numeric(df[column],errors='coerce',downcast=d_cast)
        else:
            df[column] = pd.to_numeric(df[column],errors='coerce')

### Data conversions 
Perform the following type conversions:
* Convert __release_date__ to datetime
* Convert __budget__ and __revenue__ to numerics
* Convert __runtime__ to float
* Convert __vote_average__ to float
* Convert __vote_count__ to integer
* Convert all JSON fields to abstract syntax trees

In [ ]:
# convert movies columns to numeric
def data_convert(movies_df):
    if 'release_date' in movies_df.columns:
        movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')
    if 'budget' in movies_df.columns:
        movies_df['budget']  = pd.to_numeric(movies_df['budget'],  errors='coerce')
        movies_df['budget']  = movies_df['budget'].replace(0, np.nan)
    if 'revenue' in movies_df.columns:
        movies_df['revenue']  = pd.to_numeric(movies_df['revenue'],  errors='coerce')
        movies_df['revenue']  = movies_df['revenue'].replace(0, np.nan)
    if 'runtime' in movies_df.columns:
        cast_to_float(movies_df,['runtime'])
    if 'vote_average' in movies_df.columns:
        cast_to_float(movies_df,['vote_average'])
    if 'vote_count' in movies_df.columns:
        cast_to_float(movies_df,['vote_count'])
    if 'popularity' in movies_df.columns:
        cast_to_float(movies_df,['popularity'])
  
    return movies_df

In [ ]:
movies = data_convert(movies)

__json columns__

In [ ]:
# convert json columns to abstract syntax trees
json_columns = ['belongs_to_collection',
                'genres',
                'production_companies',
                'production_countries',
                'spoken_languages',
                'cast',
                'crew',
                'keywords']

convert_json_to_ast(movies, json_columns)

In [ ]:
# convert the columns that contain the dictionary field with the name value
get_dict_val_from_ast(movies, ['belongs_to_collection'],'name','')

In [ ]:
# convert the columns that contain the list field with the name value
get_list_val_from_ast(movies, ['genres','spoken_languages'],'name','Other')
get_list_val_from_ast(movies, ['production_companies','keywords'],'name','')
get_list_val_from_ast(movies, ['production_countries'],'iso_3166_1','Other')

### Create data checkpoint - checkpoint_2
Afer data conversion, before cleaning up empty values

In [ ]:
save_checkpoint(movies, 'movies', '2')

In [ ]:
# create a name
movies.name = 'movies'

In [ ]:
# in case it's needed - restore the checkpoint
# movies = restore_checkpoint('movies','2')
# movies = data_convert(movies)

### Handle empty values

__revenue__
* As revenue is the target value in the model, we have to clear all rows with empty revenue

In [ ]:
# check shape before
print(movies.shape)

In [ ]:
# remove empty target values
movies = movies[movies['revenue'].notnull()]

In [ ]:
# check shape after
print(movies.shape)

In [ ]:
# running a quick schema and data analysis after reducing empty revenue items:
quick_schema_analysis(movies)
quick_data_analysis(movies)

__budget, runtime__  - replace nan with mean

In [ ]:
movies['budget'].fillna((movies['budget'].mean()), inplace=True)
movies['runtime'].fillna((movies['runtime'].mean()), inplace=True)

__homepage, overview, poster_path, status, tagline__ - replace nan with empty string

In [ ]:
movies['homepage'].fillna('',inplace=True)
movies['overview'].fillna('',inplace=True)
movies['poster_path'].fillna('',inplace=True)
movies['status'].fillna('',inplace=True)
movies['tagline'].fillna('',inplace=True)

### Examine distribution and correlations

In [ ]:
# define the collection of numeric columns
num_columns = ['budget','popularity','runtime','vote_average','vote_count','revenue']

In [ ]:
# plot histograms
movies[num_columns].hist(bins=50, figsize=(11,8))
plt.show()

In [ ]:
# examine correlaiton of numerical columns
scatter_matrix(movies[num_columns], figsize=(15, 15))
plt.show()

*__TO_REVIEW__* 
* outlier detection and removal

### Create data checkpoint - checkpoint_3
Before removing unnecessary features and adding new features to movies

In [ ]:
save_checkpoint(movies, 'movies', '3')

In [ ]:
# in case it's needed - restore the checkpoint
# movies = restore_checkpoint('movies','3')
# movies = data_convert(movies)

### Feature removal
Drop features that don't provide any added value

In [ ]:
movies.drop(columns = 'poster_path',inplace=True) # textual column with low added value
movies.drop(columns = 'imdb_id',inplace=True) # not used in this model

In [ ]:
# assess categorial columns
assess_columns(movies,['adult','video'],True)

In [ ]:
# drop video and adult as their distinct value is False
movies.drop(columns = ['adult','video'],inplace=True)

In [ ]:
# case for dropping original_title
compare = ['title', 'original_title']
movies[movies['original_title'] != movies['title']][['title', 'original_title']].head(5)

In [ ]:
# drop original_title (it appears to be the untranslated version of the title)
movies.drop(columns='original_title',inplace=True)

### Feature engineering

#### New feature - actors (from cast)

In [ ]:
# create actors columns based on cast
get_list_val_from_ast(df=movies, columns=['cast'],field_name='name',new_col_dict = {'cast' : 'actors'})

In [ ]:
assess_columns(movies, ['actors'])

#### New feature - director (from crew)

In [ ]:
# extract the director from the crew field
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return 'Unknown'
  
movies['director'] = movies['crew'].apply(get_director)

In [ ]:
assess_columns(movies, ['director'])

#### New features - cast_size and crew_size (from cast and crew)

In [ ]:
movies['cast_size'] = movies['cast'].apply(lambda x: len(x))
movies['crew_size'] = movies['crew'].apply(lambda x: len(x))

In [ ]:
assess_columns(movies, ['cast_size','crew_size'])

#### New feature - franchise (from belongs_to_collection)

In [ ]:
movies['franchise'] = (movies['belongs_to_collection']
                       .apply(lambda x: len(x)>0)
                      )

In [ ]:
assess_columns(movies, ['franchise'])

#### New features - season and release_year (from release_date)

In [ ]:
# calculate the season of the release (spring, summer, fall, winter)
def season_of_date(date):
    if pd.isnull(date):
        return 'unknown'
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'

# create a new column    
movies['season'] = (movies['release_date']
                          .fillna(pd.NaT)
                          .apply(lambda x: season_of_date(x))
                   )

In [ ]:
# get the year the movie was released
def year_of_date(date):
    if pd.isnull(date):
      return -1
    year = date.year
    return year

# create a new column    
movies['release_year'] = (movies['release_date']
                          .fillna(pd.NaT)
                          .apply(lambda x: year_of_date(x))
                   )

In [ ]:
assess_columns(movies, ['season','release_year'],True)

#### New feature -  home_pagedomain (from homepage)

In [ ]:
# get domain from url
def get_url_domain(url):
    if(not pd.isnull(url)):
        parsed_uri = urlparse(url )
        return '{uri.netloc}'.format(uri=parsed_uri)
    else:
        return url

In [ ]:
# extract the domain from the homepage url
movies['homepage_domain'] = movies['homepage'].apply(get_url_domain)

In [ ]:
assess_columns(movies, ['homepage_domain'])

#### New feature - weighted_rating (from vote_average and vote_count)
Weighted Rating (WR) = (vv+m.R)+(mv+m.C)(vv+m.R)+(mv+m.C) where,

v is the number of votes for the movie
m is the minimum votes required to be listed in the chart
R is the average rating of the movie C is the mean vote across the whole report.
Source: https://www.kaggle.com/rounakbanik/movie-recommender-systems

In [ ]:
# add a weighted rating feature
vote_averages = (movies[movies['vote_average']
                 .notnull()]['vote_average'].astype('int')
                )

vote_counts = (movies[movies['vote_count']
               .notnull()]['vote_count'].astype('int')
              )

C = vote_averages.mean()
m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['vote_count']+1
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

movies['weighted_rating'] = movies.apply(weighted_rating, axis=1)

In [ ]:
assess_columns(movies, ['weighted_rating'])

#### New feature - revenue_to_budget_ratio (from revenue and budget)

In [ ]:
# this new feature can indicate "success" or "failure"
# (depending on whether ration is < 0 or > 0)
movies['revenue_to_budget_ratio'] = movies['revenue'] / movies['budget']

In [ ]:
assess_columns(movies, ['revenue_to_budget_ratio'])

#### Drop columns that are no longer needed
Drop columns that were already engineered to generate other columns

In [ ]:
# drop additional columns
movies.drop(columns = 'homepage',inplace=True)  #replaced by homepage_domain
movies.drop(columns = 'cast',inplace=True)  # parsed into actors
movies.drop(columns = 'crew',inplace=True)  # partially expressed in director feature
movies.drop(columns = 'release_date',inplace=True)  # can be replaced with season and release_year

#### Examine distribution and correlations - of new features

In [ ]:
# define the collection of new numeric columns
new_num_columns = ['cast_size','crew_size','release_year','weighted_rating','revenue_to_budget_ratio']

In [ ]:
# plot histograms
movies[new_num_columns].hist(bins=50, figsize=(11,8))
plt.show()

In [ ]:
# examine correlaiton of numerical columns
if not 'revenue' in new_num_columns:
    new_num_columns.append('revenue')
scatter_matrix(movies[new_num_columns], figsize=(15, 15))
plt.show()

#### Remove columns that don't have any correlation with revenue

In [ ]:
# drop columns that are not correlated with revenue
movies.drop(columns = ['revenue_to_budget_ratio','release_year'],inplace=True)

#### Asses movies data set to apply final data cleansing

In [ ]:
movies.name = 'movies'
quick_schema_analysis(movies)

### Create data checkpoint - checkpoint_4
After feature engineering and some feature removal

In [ ]:
save_checkpoint(movies, 'movies', '4')

In [ ]:
# in case it's needed - restore the checkpoint
# movies = restore_checkpoint('movies','4')
# movies = data_convert(movies)
# movies['overview'].fillna('',inplace=True)
# movies['status'].fillna('',inplace=True)
# movies['tagline'].fillna('',inplace=True)
# movies['homepage_domain'].fillna('',inplace=True)

In [ ]:
movies.name = 'name'

In [ ]:
quick_schema_analysis(movies)

In [ ]:
# create a backup data set
movies_new = movies.copy()

In [ ]:
movies = movies_new.copy()

### Handle textual and categorical features

#### Get the stopwords file

In [ ]:
# specify file names and locations
stop_file_url = URL_GIT_DOMAIN + URL_GIT_PROJECT_PATH + '/stopwords.txt'

fetch_data(stop_file_url, 'stopwords.txt', PROJECT_LOCAL_RESOURCE_DIR, zip=False)

In [ ]:
# load stop words file
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words(os.path.join(PROJECT_LOCAL_RESOURCE_DIR,'stopwords.txt'))

#### Encoded and vectorized feature reduction functions

In [ ]:
# reduce the encoded features using a threshold
def reduce_sparse_encoding(df,threshold):
    headers_list = list(df.columns.values)
    num_samples = df.shape[0]

    for header in headers_list:
        col_true_count = len(df[df[header] == 1])
        if col_true_count/num_samples < threshold:
            df = df.drop(header, axis=1)
    
    return df

In [ ]:
# reduce the weighted features using a threshold
def reduce_sparse_weights(df,threshold):
    headers_list = list(df.columns.values)

    for header in headers_list:
        col_weight_sum = df[header].sum()
        if col_weight_sum < threshold:
            df = df.drop(header, axis=1)
    
    return df

#### Title

In [ ]:
# vectorizing the titles - using word counts
count_vectorizer = CountVectorizer(max_features=1000, binary=True, max_df=0.8,stop_words=stopwords) 
titles = movies["title"] 
titles_transformed = count_vectorizer.fit_transform(titles)

In [ ]:
# examine the vectorized titles
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
title_titles = ['title_' + title for title in idx_to_word]
titles_df = pd.DataFrame(titles_transformed.toarray(),columns=title_titles)
titles_df.shape

In [ ]:
# reduce the features using 0.1% of 1
titles_df = reduce_sparse_encoding(titles_df,0.001)
titles_df.shape

#### belongs_to_collection

In [ ]:
# replace empty lists with an empty string
collections = [''.join(collection).lower().replace('collection','').replace('series','') 
               for collection in movies['belongs_to_collection'].values]
# Vectorizing the collection names - using word counts
count_vectorizer = CountVectorizer(max_features=100, stop_words=stopwords) 
collections_transformed = count_vectorizer.fit_transform(collections)

In [ ]:
# examine the vectorized collections
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
collection_titles = ['coll_' + title for title in idx_to_word]
collections_df = pd.DataFrame(collections_transformed.toarray(),columns=collection_titles)
collections_df.shape

In [ ]:
# reduce the features using 0.1% of 1
collections_df = reduce_sparse_encoding(collections_df,0.001)
collections_df.shape

#### keywords

In [ ]:
# vectorizing keywords - re-use the vectorizer used for collections
keywords = [''.join(keyword).lower() 
               for keyword in movies['keywords'].values]

# Vectorizing the keywords - using word counts
count_vectorizer = CountVectorizer(max_features=100, stop_words=stopwords) 
keywords_transformed = count_vectorizer.fit_transform(keywords)

In [ ]:
# examine the vectorized keywords
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
keyword_titles = ['key_' + title for title in idx_to_word]
keywords_df = pd.DataFrame(keywords_transformed.toarray(),columns=keyword_titles)
keywords_df.shape

In [ ]:
# reduce the features using 0.1% of 1
keywords_df = reduce_sparse_encoding(keywords_df,0.001)
keywords_df.shape

#### tagline

In [ ]:
# vectorizing tagline - using tf-idf weighted term-document matrix
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8,stop_words=stopwords) 
taglines = movies['tagline'].replace(np.nan,'')
taglines_transformed = tfidf_vectorizer.fit_transform(taglines)

In [ ]:
# examine the vectorized taglines
idx_to_word = np.array(tfidf_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# apply NMF
nmf = NMF(n_components=100, solver="mu")
taglines_nmf = nmf.fit_transform(taglines_transformed)

In [ ]:
# create a DataFrame for tagline
tagline_titles = nmf.components_
tagline_titles = ['tagline_' + '_'.join(idx_to_word[title.argsort()[-10:]]) for title in tagline_titles]
tagline_df = pd.DataFrame(taglines_nmf,columns=tagline_titles)
tagline_df.shape

In [ ]:
# reduce the features using sum weights of 1
tagline_df = reduce_sparse_weights(tagline_df,1)
tagline_df.shape

#### Overview

In [ ]:
# vectorizing overview - using tf-idf weighted term-document matrix 
overviews = movies['overview'].replace(np.nan,'')
tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.8,stop_words=stopwords) 
overviews_transformed = tfidf_vectorizer.fit_transform(overviews)

In [ ]:
# examine the vectorized overviews
idx_to_word = np.array(tfidf_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# apply NMF : re-use the nmf component 
overviews_nmf = nmf.fit_transform(overviews_transformed)

In [ ]:
# create a DataFrame for overview
overview_titles = nmf.components_
overview_titles = ['overview_' + '_'.join(idx_to_word[title.argsort()[-10:]]) for title in overview_titles]
overview_df = pd.DataFrame(overviews_nmf,columns=overview_titles)
overview_df.shape

In [ ]:
# reduce the features using sum weights of 1
overview_df = reduce_sparse_weights(overview_df,1)
overview_df.shape

#### spoken_languages

In [ ]:
# vectorizing spoken languages - re-use the vectorizer used for collections
count_vectorizer = CountVectorizer()
spoken_laguages = [','.join(lang).lower() 
               for lang in movies['spoken_languages'].values]
spoken_laguages_transformed = count_vectorizer.fit_transform(spoken_laguages)

In [ ]:
# examine the vectorized spoken languages
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
spoken_languages_titles = ['s_lang_' + lang for lang in idx_to_word]
spoken_languages_df = pd.DataFrame(spoken_laguages_transformed.toarray(),columns=spoken_languages_titles)
spoken_languages_df.shape

In [ ]:
# reduce the features using 0.1% of 1
spoken_languages_df = reduce_sparse_encoding(spoken_languages_df,0.001)
spoken_languages_df.shape

#### original_language

In [ ]:
# vectorizing original language - re-use the vectorizer used for collections
count_vectorizer = CountVectorizer()
original_languages = movies['original_language'].replace(np.nan,'')
original_languages_transformed = count_vectorizer.fit_transform(original_languages)

In [ ]:
# examine the vectorized original languages
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
original_languages_titles = ['o_lang_' + lang for lang in idx_to_word]
original_languages_df = pd.DataFrame(original_languages_transformed.toarray(),columns=original_languages_titles)
original_languages_df.shape

In [ ]:
# reduce the features using 0.1% of 1
original_languages_df = reduce_sparse_encoding(original_languages_df,0.001)
original_languages_df.shape

#### homepage_domain

In [ ]:
# vectorizing original language - using ngram vectorizing (to select all 3 parts of the url)
ngram_vectorizer = CountVectorizer(max_features=100, binary=True, max_df=0.8,stop_words=stopwords,ngram_range=(3, 3)) 
homepage_domains = movies['homepage_domain'].replace(np.nan,'')
homepage_domains_transformed = ngram_vectorizer.fit_transform(homepage_domains)

In [ ]:
# examine the vectorized original languages
idx_to_word = np.array(ngram_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
homepage_domain_titles = ['home_'+ url.replace(' ','.') for url in idx_to_word]
homepage_domains_df = pd.DataFrame(homepage_domains_transformed.toarray(),columns=homepage_domain_titles)
homepage_domains_df.shape

In [ ]:
# reduce the features using 0.1% of 1
homepage_domains_df = reduce_sparse_encoding(homepage_domains_df,0.001)
homepage_domains_df.shape

#### genres

In [ ]:
# Prepare the data in the column
genres = [','.join(gen) for gen in movies['genres'].values]

# Vectorizing genres 
count_vectorizer = CountVectorizer()
genres_transformed = count_vectorizer.fit_transform(genres)

In [ ]:
# examine the vectorized production genres
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# set titles and define new DataFrame
genre_titles = ['gen_' + gen for gen in idx_to_word]
genres_df = pd.DataFrame(genres_transformed.toarray(),columns=genre_titles)
genres_df.shape

In [ ]:
# reduce the features using 0.1% of 1
genres_df = reduce_sparse_encoding(genres_df,0.001)
genres_df.shape

#### production_companies

In [ ]:
# prepare the data in the column
production_companies = [','.join(prod).strip().replace(' ','_') 
               for prod in movies['production_companies'].values]

# Vectorizing production companies 
count_vectorizer = CountVectorizer(max_features=1000)
production_companies_transformed = count_vectorizer.fit_transform(production_companies)

In [ ]:
# examine the vectorized production companies
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# set titles and define new DataFrame
production_companies_titles = ['prod_' + prod for prod in idx_to_word]
production_companies_df = pd.DataFrame(production_companies_transformed.toarray(),columns=production_companies_titles)
production_companies_df.shape

In [ ]:
# reduce the features using 0.1% of 1
production_companies_df = reduce_sparse_encoding(production_companies_df,0.001)
production_companies_df.shape

#### production_countries

In [ ]:
# Vectorizing production countries 
count_vectorizer = CountVectorizer() 
production_countries = [','.join(country).lower() 
               for country in movies['production_countries'].values]
production_countries_transformed = count_vectorizer.fit_transform(production_countries)

In [ ]:
# examine the vectorized production countries
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# create a DataFrame
production_countries_titles = ['country_' + country for country in idx_to_word]
production_countries_df = pd.DataFrame(production_countries_transformed.toarray(),columns=production_countries_titles)
production_countries_df.shape

In [ ]:
# reduce the features using 0.1% of 1
production_countries_df = reduce_sparse_encoding(production_countries_df,0.001)
production_countries_df.shape

#### director

In [ ]:
# prepare the data in the column
directors = [director.strip().replace(' ','_') 
               for director in movies['director'].values]

# Vectorizing production companies
count_vectorizer = CountVectorizer(max_features=100) 
directors_transformed = count_vectorizer.fit_transform(directors)

In [ ]:
# examine the vectorized directors
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# set titles and define new DataFrame
director_titles = ['director_' + director for director in idx_to_word]
directors_df = pd.DataFrame(directors_transformed.toarray(),columns=director_titles)
directors_df.shape

In [ ]:
# reduce the features using 0.1% of 1
directors_df = reduce_sparse_encoding(directors_df,0.001)
directors_df.shape

#### actors

In [ ]:
# prepare the data in the column
actors = [','.join(actor).strip().replace(' ','_') 
               for actor in movies['actors'].values]

# Vectorizing actors
count_vectorizer = CountVectorizer(max_features=1000) 
actors_transformed = count_vectorizer.fit_transform(actors)

In [ ]:
# examine the vectorized actors
idx_to_word = np.array(count_vectorizer.get_feature_names())
print(idx_to_word)

In [ ]:
# set titles and define new DataFrame
actor_titles = ['actor_' + actor for actor in idx_to_word]
actors_df = pd.DataFrame(actors_transformed.toarray(),columns=actor_titles)
actors_df.shape

In [ ]:
# reduce the features using 0.1% of 1
actors_df = reduce_sparse_encoding(actors_df,0.001)
actors_df.shape

#### one-hot encoding for categorical features

#### season

In [ ]:
season_df = pd.get_dummies(movies[['season']],prefix='season')
season_df.shape

In [ ]:
# reduce the features using 0.1% of 1
season_df = reduce_sparse_encoding(season_df,0.001)
season_df.shape

#### status

In [ ]:
status_df = pd.get_dummies(movies[['status']],prefix='status')
status_df.shape

In [ ]:
# reduce the features using 0.1% of 1
status_df = reduce_sparse_encoding(status_df,0.001)
status_df.shape

#### franchise

In [ ]:
franchise_df = pd.get_dummies(movies[['franchise']],prefix='fran')
franchise_df.shape

### Merge encoded and vectorized sets into movies

In [ ]:
remove_columns = ['revenue','title', 'belongs_to_collection', 'keywords', 'tagline', 'overview', 'spoken_languages',
                  'original_language', 'homepage_domain', 'genres', 'production_companies', 'production_countries', 
                  'director', 'actors', 'season', 'status', 'franchise']

# create the final data set
movies_final_arr = np.concatenate((movies[['revenue']].values,movies.drop(remove_columns,axis=1).values,
                                 titles_df.values,collections_df.values,
                                 keywords_df.values,tagline_df.values,overview_df.values,
                                 spoken_languages_df.values,original_languages_df.values,
                                 homepage_domains_df.values,genres_df.values,
                                 production_companies_df.values,production_countries_df.values,
                                 directors_df.values,actors_df.values,season_df.values,
                                 status_df.values,franchise_df.values),  axis=1)

# set column headers
movies_headers_arr = np.concatenate((movies[['revenue']].columns.values,movies.drop(remove_columns,axis=1).columns.values,
                                 titles_df.columns.values,collections_df.columns.values,
                                 keywords_df.columns.values,tagline_df.columns.values,overview_df.columns.values,
                                 spoken_languages_df.columns.values,original_languages_df.columns.values,
                                 homepage_domains_df.columns.values,genres_df.columns.values,
                                 production_companies_df.columns.values,production_countries_df.columns.values,
                                 directors_df.columns.values,actors_df.columns.values,season_df.columns.values,
                                 status_df.columns.values,franchise_df.columns.values))

# create a DataFrame object
movies_final = pd.DataFrame(movies_final_arr,columns=movies_headers_arr)

In [ ]:
# check the new schema
quick_schema_analysis(movies_final)

In [ ]:
# assign back to movies
movies = movies_final

### Create data checkpoint - checkpoint_5
After creating the full set of the features 

In [ ]:
save_checkpoint(movies, 'movies', '5')

In [ ]:
# in case it's needed - restore the checkpoint
# movies = restore_checkpoint('movies','5')

### Feature selction and reduction - continued
Once the full feature set has been assembled, we use feature selection and reduction algorithms to come up with a subset of the features

The algorithms below is optimized for regression (specifically linear regression). Regardless, further algorithms we'll be used in the model creation section

In [ ]:
# define target value and features
target = 'revenue'
features = list(movies.columns)
features = [f for f in features if f!=target]

# create data 
feature_set = movies[features]
target_set = movies[target]

In [ ]:
#add a new target variable for classification - revenue was over 2.5 times the budget
target_new = np.ravel(target_set)/np.ravel(feature_set['budget'])>2.50

In [ ]:
# create train and test sets
X_tr, X_te, y_tr, y_te = train_test_split(feature_set, target_set, test_size=0.3)

In [ ]:
#add a new target variable for classification 
# based on revenue to budget ratio (set to true if > 2.5)
y_tr_cl, y_te_cl = train_test_split(target_new, test_size=0.3)

#### Scoring functions

In [ ]:
# display the score
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())

#### SelectKBest with chi square

In [ ]:
# define the select K best model - using chi square
select_best = SelectKBest(score_func=chi2)

In [ ]:
# identify the best hyper parameter
k_arr = [20, 30, 40 ,50, 75, 100, 200]

for k in k_arr:
    select_best.k = k
    select_best.fit(np.asarray(X_tr),np.asarray(y_tr, dtype="|S100"))
    X_tr_new = select_best.transform(X_tr)
    X_te_new = select_best.transform(X_te)
    print("\n")
    print("Performance for k={}".format(k))
    print("Trainng set:")
    lin_scores = cross_val_score(LinearRegression(), X_tr_new, y_tr, scoring="neg_mean_squared_error", cv=4)
    lin_rmse_scores = np.sqrt(-lin_scores)
    display_scores(lin_rmse_scores)
    print("Test set:")
    lin_scores = cross_val_score(LinearRegression(), X_te_new, y_te, scoring="neg_mean_squared_error", cv=4)
    lin_rmse_scores = np.sqrt(-lin_scores)
    display_scores(lin_rmse_scores)

* __Conclusion:__ Best test results are for k=75

In [ ]:
# pick the best : for k=75
select_best.k = 75
select_best.fit(np.asarray(X_tr),np.asarray(y_tr, dtype="|S100"))
X_tr = select_best.transform(X_tr)
X_te = select_best.transform(X_te)

#### Scaling the features

In [ ]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)
X_te = scaler.transform(X_te)

#### Feature tuning pipeline (using linear regression)

In [ ]:
# Linear regression and pipeline (to evaluate feature selection and reduction)
steps = [('regression',LinearRegression())]
pipeline = Pipeline(steps)

#### PCA

In [ ]:
# Make an instance of the Model
pca = PCA()

In [ ]:
# add to the steps collection
pipeline.steps.insert(0,('pca',pca))

In [ ]:
# GridSearchCV
parameters = {}
grid_search = GridSearchCV(pipeline,parameters, cv=4, scoring='neg_mean_squared_error')

In [ ]:
# add corresponding parameters
grid_search.param_grid['pca__n_components'] = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,0.999,1]

In [ ]:
# fit the model
grid_search.fit(X_tr,y_tr)

In [ ]:
# training scores
print("Best pca parameter is {}".format(grid_search.best_params_))
lin_scores = grid_search.best_score_
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# run best model with test data
grid_search_final = grid_search.best_estimator_ 
grid_search_final.fit(X_te,y_te)
lin_scores = cross_val_score(grid_search_final, X_te, y_te, scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

* __Conclusion__: PCA doesn't improve the results

In [ ]:
# tidy up - remove the pca step from the pipeline
del pipeline.steps[0]

#### Clustering - K-means

In [ ]:
kmeans = KMeans()

# add to the steps collection
pipeline.steps.insert(0,('kmeans',kmeans))

In [ ]:
# GridSearchCV
parameters = {}
grid_search = GridSearchCV(pipeline,parameters, cv=4, scoring='neg_mean_squared_error')

# add corresponding parameters
grid_search.param_grid['kmeans__n_clusters'] = [2,4,6,8,10,15,20,30,40]

In [ ]:
# fit the model
grid_search.fit(X_tr,y_tr)

In [ ]:
# training set scores
print("Best k-means parameter is {}".format(grid_search.best_params_))
lin_scores = grid_search.best_score_
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
# try with test data
grid_search_final = grid_search.best_estimator_ 
grid_search_final.fit(X_te,y_te)
lin_scores = cross_val_score(grid_search_final, X_te, y_te, scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

* __Conclusion__: KMeans clustering doesn't improve the results

## 4. Model

### Reasons for considering a classification model in addition to the regression model.

Box Office profits are a function of production budgets, ticket prices & Marketing/ Distribution costs based on their year of release.


Hence, while evaluating historical values, profit calculations without accounting for inflation doesnt always provide the big picture (pun intended). A recommended option is arriving at a revenue to budget ratio (R/B), which offers better insights compared to revenue recognition alone. 

Further, this ratio is also useful in determining the relative success of a theatrical release. The total cost of movie production is not limited to production budgets but should take into account the distribution and marketing costs which is increasingly becoming important. Accordingly, movie producers and industry watchers now believe that a movie needs to make in excess of 2.5 times its production budget even to be considered a moderate success.

Hence, the classifier feature matrix can drop the 'revenue' and 'revenue to budget ratio' as new unseen data ( data on movies that are yet to be released which we want to know whether will be a success or failure) will not have a revenue column. The attempt here is to predict film success using features like genre, production company, month/season of release, cast/crew size, country of production, whether part of a franchise, weighted rating etc. These are parameters that are already known before the movie's release.

### Classification

#### Logistic regression

In [ ]:
logreg=LogisticRegression()
params_logreg={'C':np.logspace(-5, 8, 15)}
logreg_cv = GridSearchCV(logreg,params_logreg,cv=4)
logreg_cv.fit(X_tr,y_tr_cl)

In [ ]:
print('Accuracy: ',logreg_cv.best_score_)

#### Random forest

In [ ]:
forest=RandomForestClassifier(random_state=42)
params_forest = {'n_estimators': [3, 4, 6, 7, 10, 20, 50, 100]}
forest_cv=GridSearchCV(forest, params_forest,cv=4)
forest_cv.fit(X_tr,y_tr_cl)

In [ ]:
print('Accuracy: ',forest_cv.best_score_)

#### KNN

In [ ]:
knn=KNeighborsClassifier()
params_knn = {'n_neighbors': [3, 5, 10, 20]}
knn_cv=GridSearchCV(knn, params_knn,cv=4,n_jobs=-1)
knn_cv.fit(X_tr,y_tr_cl)

In [ ]:
print('Accuracy:',knn_cv.best_score_)

#### Decision tree

In [ ]:
tree = DecisionTreeClassifier()
params_tree = {"criterion": ["gini", "entropy"],"min_samples_split": [2, 10, 20],"max_depth": [None, 2, 5, 10],
              "min_samples_leaf": [1, 5, 10],"max_leaf_nodes": [None, 5, 10, 20]
              }
tree_cv=GridSearchCV(tree, params_tree,cv=4,n_jobs=-1)
tree_cv.fit(X_tr,y_tr_cl)

In [ ]:
print('Accuracy:',tree_cv.best_score_)

#### AdaBoostClassifier with DecisionTree as base estimator

In [ ]:
ABC= AdaBoostClassifier(base_estimator=tree_cv.best_estimator_,n_estimators=100)
np.mean(cross_val_score(ABC, X_tr, y_tr_cl, cv=3))

#### Voting classifier

In [ ]:
classifiers = [('Logistic Regression', logreg_cv.best_estimator_),('K Nearest Neighbours', knn_cv.best_estimator_),
               ('RandomForestClassifier', forest_cv.best_estimator_),('DecisionTreeClassifier',tree_cv.best_estimator_)]

vc = VotingClassifier(estimators=classifiers)

In [ ]:
print('Accuracy: ',np.mean(cross_val_score(vc, X_tr, y_tr_cl, cv=3)))

### Regression

#### Linear regression

In [ ]:
lin_reg=LinearRegression()
lin_scores = cross_val_score(lin_reg,X_tr,y_tr, scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

#### Ridge regression

In [ ]:
param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000,1000]}]
rr_cv = GridSearchCV(Ridge(), param_grid, cv=4, scoring='neg_mean_squared_error')
rr_cv.fit(X_tr, y_tr)

In [ ]:
print(np.sqrt(-rr_cv.best_score_))

#### Lasso regression

In [ ]:
param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000,1000]}]
lr_cv = GridSearchCV(Lasso(), param_grid, cv=3, scoring='neg_mean_squared_error')
lr_cv.fit(X_tr, y_tr)

In [ ]:
print(np.sqrt(-lr_cv.best_score_))

### Test data set evaluation

#### Classification

In [ ]:
final_cl_model = logreg_cv.best_estimator_  
y_pred_cl = final_cl_model.predict(X_te)

print('Accuracy score: ',accuracy_score(y_te_cl,y_pred_cl))

* Logistic Regression has a train accuracy of __0.769__ and a test accuracy of __0.782__

#### Regression

In [ ]:
final_reg_model = lin_reg
final_reg_model.fit(X_tr,y_tr)

y_pred_reg = final_reg_model.predict(X_te)

final_mse = mean_squared_error(y_te, y_pred_reg)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

### Model evaluation

#### Evaluation functions

In [ ]:
# hitmap for the confusion matrix
def cm_heatmap(y_test,y_pred):
    cm=confusion_matrix(y_test,y_pred)
    conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
    plt.figure(figsize = (8,5))
    sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

In [ ]:
# compare different models
def metrics_comp(y_te_cl,y_pred_cl,y_pred_forest):
    metrics=pd.DataFrame(index=['Accuracy','Precision','Recall'],columns=['Logistic Regression','Random Forest'])
    metrics.loc['Accuracy','Logistic Regression']=accuracy_score(y_te_cl,y_pred_cl)
    metrics.loc['Precision','Logistic Regression']=precision_score(y_te_cl,y_pred_cl)
    metrics.loc['Recall','Logistic Regression']=recall_score(y_te_cl,y_pred_cl)
    metrics.loc['Accuracy','Random Forest']=accuracy_score(y_te_cl,y_pred_forest)
    metrics.loc['Precision','Random Forest']=precision_score(y_te_cl,y_pred_forest)
    metrics.loc['Recall','Random Forest']=recall_score(y_te_cl,y_pred_forest)
    return(metrics)

In [ ]:
# precison-recall graph
def p_r_threshold(thresholds,precision,recall):
    fig,ax=plt.subplots(figsize=(8,5))
    ax.plot(thresholds_logreg,precision_logreg[1:],label='Precision')
    ax.plot(thresholds_logreg,recall_logreg[1:],label='Recall')
    ax.set_xlabel('Classification threshold')
    ax.set_ylabel('precision,Recall')
    ax.set_title('Logistic Regression: Precison Recall')
    ax.legend()
    ax.grid();

#### Best model - logistic regression

#### Confusion matrix

In [ ]:
cm_heatmap(y_te_cl,y_pred_cl)

#### ROC curve and AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_te_cl, final_cl_model.predict_proba(X_te)[:,1])
plt.plot(fpr,tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for RandomForest classifier')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('Recall')
plt.grid(True)

In [ ]:
print ("Area under the curve is ", roc_auc_score(y_te_cl,final_cl_model.predict_proba(X_te)[:,1]))

#### Evaluaiton of the 2nd best model

In [ ]:
# Test data
y_pred_forest = forest_cv.predict(X_te)
print('Accuracy score: ',accuracy_score(y_te_cl,y_pred_forest))

In [ ]:
# Confusion matrix
cm_heatmap(y_te_cl,y_pred_forest)

In [ ]:
print(classification_report(y_te_cl, y_pred_forest))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_te_cl, forest_cv.predict_proba(X_te)[:,1])
plt.plot(fpr,tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for RandomForest classifier')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('Recall')
plt.grid(True)

In [ ]:
print ("Area under the curve is ", roc_auc_score(y_te_cl,forest_cv.predict_proba(X_te)[:,1]))

### Comparision of Accuracy, Precision and Recall between Logistic Regression and Random Forest classifiers
In the case of movie box office prediction, a high recall translates to missing to identify a film that will be successful and high precision ensures the majority of positively predicted films will be successful.

As the average cost involved in producing a major studio film is extremely high and the number of films produced by a production house is relatively low, it is important that the classifier's positive predictions are going to be successful. Failing that will result in huge loss for the production house. Hence a model that has high precision rather than high recall is preferred.

In [ ]:
# compare logistic regression and random forrest
metrics_comp(y_te_cl,y_pred_cl,y_pred_forest)

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
metrics_comp(y_te_cl,y_pred_cl,y_pred_forest).plot(kind='bar',ax=ax)
plt.show()

* __Conclusion__ :Accuracy levels are consistant between both models. However, Logistic regression has higher precision than Random forest.

#### Tradeoff between precision and recall at different thresholds

In [ ]:
precision_logreg, recall_logreg, thresholds_logreg=precision_recall_curve(y_te_cl,final_cl_model.predict_proba(X_te)[:,1])

In [ ]:
p_r_threshold(thresholds_logreg,precision_logreg,recall_logreg)

* __Conclusion__: At threshold 0.5 the recall is 0.4 with a very high precision (0.7). Moving the threshold to 0.6 results in a much higher precision (0.8) without a significant drop in recall.

## 5. Analysis

### Plots

### Scoring

## 6. Summary

### Observations

### Conclusions